# Lineal Regresion

## Key formulas

**y = mx1 + b**

**y = mx1 + mx2 + mx3 + ... + mxn + b**

In [ ]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt

# ==============================
# Cargar dataset
# ==============================
house_prices_ds = pd.read_csv('../../../datasets/raw/hyderabad_house_price_processed.csv')

# Usamos y=Price y features=["Area"]
X = house_prices_ds[["Area"]].dropna().values  # (n,1)
y = house_prices_ds["Price"].dropna().values   # (n,)

def draw_line(slope, intercept, color="grey", linewidth=0.7, starting=None, ending=None):
    if starting is None: 
        starting = X.min()
    if ending is None:
        ending = X.max()
    xs = np.linspace(starting, ending, 200)
    plt.plot(xs, intercept + slope*xs, linestyle="-", color=color, linewidth=linewidth)

def plot_points():
    plt.scatter(X.ravel(), y, alpha=0.5)
    plt.xlabel("Area (scaled)")
    plt.ylabel("Price (scaled)")

# ==============================
# Algoritmo Square Trick (1D)
# ==============================
def square_trick(b, w, feature, label, lr):
    prediction = w * feature + b
    error = label - prediction
    b += lr * error
    w += lr * error * feature
    return w, b

def linear_regression_interactive(X, y, lr=0.01, epochs=1000):
    w = random.random()
    b = random.random()
    history = []

    auto = False
    for epoch in range(epochs):
        i = random.randint(0, len(X)-1)
        xi = X[i][0]
        yi = y[i]

        w, b = square_trick(b, w, xi, yi, lr)
        history.append((w, b, xi, yi))

        # Graficar
        plt.figure(figsize=(7,5))
        plot_points()

        # Dibujar todas las líneas pasadas en gris
        for (wj, bj, _, _) in history[:-1]:
            draw_line(wj, bj, color="lightgrey", linewidth=0.7)

        # Última línea en rojo
        draw_line(w, b, color="red", linewidth=2)

        # Punto usado en azul
        plt.scatter(xi, yi, color="blue", s=80, zorder=5, label="Sample usado")
        plt.legend()
        plt.title(f"Epoch {epoch} - w={w:.4f}, b={b:.2f}")
        plt.show()

        if not auto:
            action = input("Presiona Enter para continuar, 'a' para auto-run hasta el final: ")
            if action.strip().lower() == "a":
                auto = True

    print("\nResultado final:")
    print(f"w={w}, b={b}")

    # ==============================
    # Dibujar línea final
    # ==============================
    plt.figure(figsize=(7,5))
    plot_points()
    draw_line(w, b, color="red", linewidth=2)
    plt.title(f"Resultado final - w={w:.4f}, b={b:.2f}")
    plt.show()

    return w, b

# ==============================
# Ejecutar
# ==============================
w, b = linear_regression_interactive(X, y, lr=0.01, epochs=1000)